In [17]:
import pandas as pd
import requests
import re
import numpy as np
from tensorflow.keras.models import *
from keras import callbacks
from tensorflow.keras.layers import *
from keras.layers import *
import random
from PIL import Image
import csv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from itertools import product
import numpy as np
import os
import torch
import torchvision
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
import torch.optim as optim
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import balanced_accuracy_score

# begin_time = "2023-09-09 18:55:00"
# end_time = "2023-09-09 19:02:06"
# ap = {
#     "apicode": "5fb9d287",
#     "begin_time": begin_time,
#     "end_time": end_time
# }

# 数据预处理

In [12]:
def segment_activities(df_s):
    activitiesSeq = []
    ponentialIndex = df_s.label.ne(df_s.label.shift())
    ii = np.where(ponentialIndex == True)[0]
    for i_s, end in enumerate(ii):
        if i_s > 0:
            df_stmp = df_s[ii[i_s - 1]:end]
            activitiesSeq.append(df_stmp)
    return activitiesSeq

def sequences_to_sentences(activity_sequences_s):
    sentences_s = []
    label_sentences_s = []
    for i_s in range(len(activity_sequences_s)):
        sentence = generate_sentence(activity_sequences_s[i_s])
        sentences_s.append(sentence)
        label_sentences_s.append(activity_sequences_s[i_s].label.values[0])
    return sentences_s, label_sentences_s

def generate_sentence(df2):
    sentence = ""
    sensors = df2.sensor.values
#    values = df2.status.values
    for i_s in range(len(sensors)):
#        val = values[i_s]
        if i_s == len(sensors) - 1:
            sentence += "{}".format(sensors[i_s])
        else:
            sentence += "{} ".format(sensors[i_s])
    return sentence

def sliding_window(sequence, win_size_s, step_s=1):
    try:
        iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    numOfChunks = int(((len(sequence) - win_size_s) / step_s) + 1)

    if win_size_s > len(sequence):
        yield sequence[0:len(sequence)]
    else:
        for i_s in range(0, numOfChunks * step_s, step_s):
            yield sequence[i_s:i_s + win_size_s]



df = pd.read_csv('linyu.csv', header=None, names=["date", "time", "sensor", "status"])
df = df.iloc[::-1].reset_index(drop=True)

# 定义特殊的sensor值
special_sensors = ['客厅', '餐厅', '入口']

# 对每个特殊的sensor进行处理
for sensor in special_sensors:
    # 新增一列，并为符合条件的行填充对应的值
    df[sensor] = (df["sensor"] == sensor) * df["status"]

print(df[:20])
# 定义一个函数来填充1和0.5之间的值
def fill_between_values(col, df, col_name):
    first_one_index = col[col == 1].index.min()
    # 如果存在1的值，将其之前的所有值填充为0.5
    if first_one_index is not None:
        df.loc[:first_one_index - 1, col_name] = 0.5

    start_index = None
    previous_value = None
    for idx, value in col.items():
        if value == 1 or value == 0.5:
            if start_index is not None:
                if previous_value == 1:
                    df.loc[start_index:idx-1, col_name] = 1
                elif previous_value == 0.5:
                    df.loc[start_index:idx-1, col_name] = 0.5
                start_index = None
            previous_value = value
        elif start_index is None:
            start_index = idx

    return df[col_name]

for col_name in df.columns[-3:]:
    df[col_name] = fill_between_values(df[col_name], df, col_name)
    
combinations = {
    (0.5, 0.5, 0.5): '0',
    (1, 0.5, 0.5): '1',
    (0.5, 1, 0.5): '2',
    (0.5, 0.5, 1): '3',
    (1, 1, 0.5): '4',
    (1, 0.5, 1): '5',
    (0.5, 1, 1): '6',
    (1, 1, 1): '7',
}

# def generate_combinations(values, num_positions):
#     # 使用itertools.product生成所有的组合
#     all_combinations = list(product(values, repeat=num_positions))
    
#     # 转换为"{0.5,0.5,0.5}"的格式，并作为标签
#     formatted_combinations = {f"{{{','.join(map(str, combo))}}}": f"{{{','.join(map(str, combo))}}}" for combo in all_combinations}
#     return formatted_combinations

# values = [1, 0.5]
# num_positions = 3
# combinations = generate_combinations(values, num_positions)

print(combinations)

# 应用转换字典
df['label'] = df.apply(lambda row: combinations.get((row['客厅'], row['餐厅'], row['入口']), 'none'), axis=1)

df = df[~df['sensor'].isin(['客厅', '餐厅', '入口'])]
df = df.drop(columns=['客厅', '餐厅', '入口'])
#print(df[:50])

#  Segment dataset in sequence of activity ##
print("STEP 3: segment dataset in sequence of activity")
activity_sequences = segment_activities(df)
print(activity_sequences[:10])

#  Transform sequences of activity in sentences ##
print("STEP 4: transform sequences of activity in sentences")
sentences, label_sentences = sequences_to_sentences(activity_sequences)
print(sentences[:10])
print(label_sentences[:10])

#  Indexization ##
print("STEP 5: sentences indexization")
# 转换文本为整数序列
sequences = [list(map(int, s.split())) for s in sentences]
#sequences = [np.array(seq) for seq in sentences]

print(sequences[:5])

#  Split in sliding windows ##
print("STEP 6: split indexed sentences in sliding windows")
X_windowed = []
Y_windowed = []
X_windowed_sen = []
Y_windowed_sen = []
win_size = 8
step = 1
for i, s in enumerate(sequences):
    chunks = sliding_window(s, win_size, step)
    for chunk in chunks:
        X_windowed.append(chunk)
        Y_windowed.append(label_sentences[i])
print(X_windowed[0:10])
print(Y_windowed[0:10])

#  Pad windows ##
print("STEP 7: pad sliding windows")
padded_windows = pad_sequences(X_windowed, padding ='post')
Y_windowed = np.array(Y_windowed)
print(padded_windows[0:10])
print(Y_windowed[0:10])

#  Save files ##
print("STEP 8: save sliding windows and labels")
# np.save("{}_{}_padded_x_step1_state_linyu.npy".format(r'/Users/zehaokou/Desktop/Technion/AI/plot/Linyu_dataset/', win_size), padded_windows)
# np.save("label_sentences.npy".format(r'/Users/zehaokou/Desktop/Technion/AI/plot/Linyu_dataset', win_size), Y_windowed)
# pickle_file_path = '/Users/zehaokou/Desktop/Technion/AI/plot/Linyu_dataset/indexed_sentences.pkl'
# with open(pickle_file_path, 'wb') as pickle_file:
#     pickle.dump(indexed_sentences, pickle_file)

          date      time sensor  status   客厅   餐厅   入口
0   2023-09-09  18:55:12      6     1.0  0.0  0.0  0.0
1   2023-09-09  18:55:12      4     1.0  0.0  0.0  0.0
2   2023-09-09  18:55:12      5     1.0  0.0  0.0  0.0
3   2023-09-09  18:55:13      3     1.0  0.0  0.0  0.0
4   2023-09-09  18:55:13      2     1.0  0.0  0.0  0.0
5   2023-09-09  18:55:14     客厅     1.0  1.0  0.0  0.0
6   2023-09-09  18:55:17     44     1.0  0.0  0.0  0.0
7   2023-09-09  18:55:28      6     0.5  0.0  0.0  0.0
8   2023-09-09  18:55:28      5     0.5  0.0  0.0  0.0
9   2023-09-09  18:55:29      6     1.0  0.0  0.0  0.0
10  2023-09-09  18:55:29      2     0.5  0.0  0.0  0.0
11  2023-09-09  18:55:32     44     0.5  0.0  0.0  0.0
12  2023-09-09  18:55:34      4     0.5  0.0  0.0  0.0
13  2023-09-09  18:55:37      2     1.0  0.0  0.0  0.0
14  2023-09-09  18:55:37      4     1.0  0.0  0.0  0.0
15  2023-09-09  18:55:38     44     1.0  0.0  0.0  0.0
16  2023-09-09  18:55:54     44     0.5  0.0  0.0  0.0
17  2023-0

# 准备npy数据

In [11]:
x = padded_windows

y = Y_windowed
y = y.astype(str)

activities = {'0': [], '1': [], '2': [], '3': [], '4': [], '5': [], '6': [], '7': []}

for i, value in enumerate(y):
    if value in activities:
        activities[value].append(i)

activity0 = activities['0']
activity1 = activities['1']
activity2 = activities['2']
activity3 = activities['3']
activity4 = activities['4']
activity5 = activities['5']
activity6 = activities['6']
activity7 = activities['7']

        
zero = np.zeros((50,50))

for i in range(1000):
    for j in range(8):
        # 如果是nan，则跳过此次循环
        if np.isnan(x[activity1[i]][j]):
            continue
            
        index = 49 - x[activity1[i]][j]
        if index != 0:
            if index - index.astype(int) != 0:
                index = index.astype(int)
                zero[index][2*j:(2*j+1)] = 0.5
            elif index - index.astype(int) == 0:
                index = index.astype(int)
                zero[index][2*j:(2*j+1)] = 1
            else:
                zero[index][2*j:(2*j+1)] = 0
                
    #计算这个窗口中不为nan的数的个数
    not_nan_count = np.sum(~np.isnan(x[activity1[i]]))
   #print(not_nan_count)
  
    for row in range(50):
        # 找到当前纵坐标下所有值不为0的点的横坐标
        cols_with_value = [col for col in range(50) if zero[row][col] > 0]
        
        if len(cols_with_value) > 0:
            # 如果第一个值是0.5，从y=0向这一点连线
            if zero[row][cols_with_value[0]] == 0.5:
                zero[row][0:cols_with_value[0]] = 1

            # 对相邻的on和off之间进行填充
            for k in range(0, len(cols_with_value) - 1):
                # If the current point is 'on' and the next point is 'off', fill between them
                if zero[row][cols_with_value[k]] == 1 and zero[row][cols_with_value[k + 1]] == 0.5:
                    zero[row][cols_with_value[k]:cols_with_value[k + 1]] = 1

            # 如果最后一个值为1，从这点向y=not_nan_count*2 - 1连线
            if zero[row][cols_with_value[-1]] == 1:
                zero[row][cols_with_value[-1]:2*not_nan_count-1] = 1
        
    for row in range(50):
# 找到当前纵坐标下所有值为1的点的横坐标
        cols_with_value = [col for col in range(50) if zero[row][col] == 1]

        # 将除了值为1的点外的其他点设置为0
        zero[row, :] = 0
        for col in cols_with_value:
            zero[row, col] = 1
            
    # 将zero数组的值乘以255
    data = zero * 255
    # 将data数组的数据类型转换为'uint8'
    data = data.astype('uint8')

    np.save("/images/1/%d.npy" % (i),data)

    # 重置zero数组为224x224的零数组，为下一次迭代做准备
    zero = np.zeros((50, 50))

print('done')

# 生成txt文件

In [ ]:
dataset_path = "/images"
num_classes = 8

# 获取每个类别的路径和图像文件名列表
class_paths = []
for i in range(num_classes):
    class_path = os.path.join(dataset_path, "{}".format(i))
    class_files = [f for f in os.listdir(class_path) if "DS" not in f]
    class_paths.append((class_path, class_files))

# 打乱每个类别的图像列表
for class_path, class_files in class_paths:
    random.shuffle(class_files)

# 将数据集分成训练集和测试集
train_file = open("train.txt", "w")
test_file = open("test.txt", "w")
for i, (class_path, class_files) in enumerate(class_paths):
    num_train = int(len(class_files) * 0.8)
    for j, filename in enumerate(class_files):
        if j < num_train:
            train_file.write("{} {}\n".format(os.path.join(class_path, filename), i))
        else:
            test_file.write("{} {}\n".format(os.path.join(class_path, filename), i))
train_file.close()
test_file.close()